Import all the dependencies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import purpleair
import pandas as pd
from purpleair.network import SensorList
from purpleair.sensor import Sensor

Pull all the current data into a data frame

In [2]:
from purpleair.network import SensorList
p = SensorList()
df = p.to_dataframe(sensor_filter='all',
                    channel='parent')
df

Initialized 22,531 sensors!


,parent,lat,lon,name,location_type,pm_2.5,temp_f,temp_c,humidity,pressure,...,last_update_check,created,uptime,is_owner,10min_avg,30min_avg,1hour_avg,6hour_avg,1day_avg,1week_avg
id,,,,,,,,,,,,,,,,,,,,,
14633,None,37.275561,-121.964134,Hazelwood canary,outside,13.64,62.0,16.666667,46.0,1012.14,...,None,None,None,False,13.70,16.01,14.32,14.14,15.27,9.80
25999,None,30.053808,-95.494643,Villages of Bridgestone AQI,outside,26.11,63.0,17.222222,69.0,1016.44,...,None,None,None,False,26.58,25.59,22.78,19.42,19.43,15.77
14091,None,37.883620,-122.070087,WC Hillside,outside,14.46,61.0,16.111111,51.0,1007.45,...,None,None,None,False,19.73,15.41,13.41,16.98,23.05,16.85
108226,None,38.573703,-121.439113,"""C"" Street Air Shelter",inside,9.15,77.0,25.000000,45.0,1020.49,...,None,None,None,False,8.88,8.44,7.88,6.78,7.18,5.33
49409,None,18.759182,99.017172,"""First's Place""",outside,30.14,98.0,36.666667,25.0,981.70,...,None,None,None,False,28.22,25.23,23.94,22.90,23.35,26.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64085,None,36.785883,127.157040,청룡동행정복지센터,outside,13.12,42.0,5.555556,31.0,1018.29,...,None,None,None,False,12.69,11.75,11.12,17.86,33.84,41.32
64995,None,36.691324,126.585255,한서대학교,outside,11.79,44.0,6.666667,35.0,1009.65,...,None,None,None,False,11.88,11.64,11.43,14.58,21.17,26.63
64093,None,36.710720,126.548390,해미읍성,outside,11.63,42.0,5.555556,39.0,1020.72,...,None,None,None,False,11.54,11.34,11.17,14.72,24.52,29.67


Code for function

In [41]:
# This is a function that takes in a list of sensor name strings and returns a data frame with data 
# from the past week for each sensor

from datetime import datetime
from datetime import timedelta

def time_series_name(sensor_list):
    data = pd.DataFrame(columns=['created_at'])
    for sensor in sensor_list:
        sensor_index = df.index[df['name'] == sensor].to_list()[0]
        se = Sensor(sensor_index)
        sensor_data = se.parent.get_historical(weeks_to_get=1,
                              thingspeak_field='secondary')[['created_at', '2.5um/dl']]
        sensor_data['created_at'] = sensor_data['created_at'].dt.floor('T')
        sensor_data = sensor_data.rename(columns={'2.5um/dl':(sensor +  ' 2.5um/dl')})
        # for cases where there is no data 
#         if len(sensor_data) < 5000:
#             continue 
        # for cases that begin at an even time 
        if (sensor_data.iloc[0]['created_at'].minute / 2) == 0:
            new_time = []
            for i in sensor_data['created_at']:
                new_time.append(i + timedelta(minutes=1))
            sensor_data['created_at'] = new_time
        data = pd.merge(data,sensor_data,how='outer',on='created_at')
        #columns = ['created_at'] + sensor_list
        #data = data.set_axis(columns, axis=1, inplace=False)
    return data[~data.isnull().any(axis=1)]



# This function is the same as the one above, but takes in a list of sensor index number rather than name strings
# This version was made because the name strings can be a little finicky about spaces and such.

def time_series_index(sensor_index_list):
    data = pd.DataFrame(columns=['created_at'])
    for sensor in sensor_index_list:
        se = Sensor(sensor)
        sensor_data = se.parent.get_historical(weeks_to_get=1,
                              thingspeak_field='secondary')[['created_at', '2.5um/dl']]
        sensor_data['created_at'] = sensor_data['created_at'].dt.floor('T')
        sensor_data = sensor_data.rename(columns={'2.5um/dl':(str(sensor) +  ' 2.5um/dl')})
        # for cases where there is no data 
#         if len(sensor_data) < 5000:
#             continue 
        # for cases that begin at an even time 
        if (sensor_data.iloc[0]['created_at'].minute / 2) == 0:
            new_time = []
            for i in sensor_data['created_at']:
                new_time.append(i + timedelta(minutes=1))
            sensor_data['created_at'] = new_time
        data = pd.merge(data,sensor_data,how='outer',on='created_at')
    return data[~data.isnull().any(axis=1)]

Testing function

In [42]:
test = time_series_name(["PA_II_D8B6", ' Hazelwood canary ', ' WC Hillside'])
test

,created_at,PA_II_D8B6 2.5um/dl,Hazelwood canary 2.5um/dl,WC Hillside 2.5um/dl
0,2021-11-23 00:01:00+00:00,1.37,1.69,23.21
1,2021-11-23 00:03:00+00:00,1.76,2.07,23.80
2,2021-11-23 00:05:00+00:00,0.31,2.34,24.18
3,2021-11-23 00:07:00+00:00,2.36,1.56,25.96
4,2021-11-23 00:09:00+00:00,2.21,1.18,18.81
...,...,...,...,...
4816,2021-11-29 23:41:00+00:00,1.97,4.52,4.89
4817,2021-11-29 23:45:00+00:00,2.81,3.25,5.44
4818,2021-11-29 23:51:00+00:00,0.81,4.50,6.00
4819,2021-11-29 23:53:00+00:00,3.11,3.81,3.69


In [38]:
test2 = time_series_index([27395, 14633, 14091, 25999])
test2

,created_at,27395 2.5um/dl,14633 2.5um/dl,14091 2.5um/dl,25999 2.5um/dl
0,2021-11-23 00:01:00+00:00,1.37,1.69,23.21,3.69
1,2021-11-23 00:03:00+00:00,1.76,2.07,23.80,6.15
2,2021-11-23 00:05:00+00:00,0.31,2.34,24.18,6.92
3,2021-11-23 00:07:00+00:00,2.36,1.56,25.96,4.88
4,2021-11-23 00:09:00+00:00,2.21,1.18,18.81,5.98
...,...,...,...,...,...
4816,2021-11-29 23:41:00+00:00,1.97,4.52,4.89,2.10
4817,2021-11-29 23:45:00+00:00,2.81,3.25,5.44,1.95
4818,2021-11-29 23:51:00+00:00,0.81,4.50,6.00,3.37
4819,2021-11-29 23:53:00+00:00,3.11,3.81,3.69,1.88


Testing with larger sensor list

In [8]:
time_series_index([98139, 97279, 27823, 99769, 99797, 98633, 51697, 3298, 3033, 50567, 27905, 116073, 86003, 31035])

,created_at,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y
0,2021-11-23 00:01:00+00:00,2.82,5.09,0.00,7.07,12.47,5.19,3.43,6.43,8.28,2.98,0.0,9.72
26,2021-11-23 00:53:00+00:00,4.26,15.11,0.00,19.55,20.84,4.80,3.28,6.78,8.24,4.45,0.0,14.07
42,2021-11-23 01:25:00+00:00,4.79,15.45,0.22,7.52,22.07,4.21,4.76,4.61,7.45,3.95,0.0,18.11
44,2021-11-23 01:29:00+00:00,4.71,14.18,0.24,13.59,23.09,6.22,4.78,2.95,6.75,8.61,0.0,17.79
46,2021-11-23 01:33:00+00:00,3.11,16.35,0.00,13.82,26.11,5.67,2.98,5.02,6.61,4.38,0.0,21.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,2021-11-23 08:39:00+00:00,8.79,23.15,0.11,36.60,16.71,13.49,3.61,10.74,11.52,9.54,0.0,27.49
263,2021-11-23 08:47:00+00:00,9.15,27.28,0.08,39.79,9.59,11.89,7.61,8.89,11.21,20.80,0.0,27.41
267,2021-11-23 08:55:00+00:00,4.79,24.18,0.64,30.18,12.66,15.55,5.82,12.37,9.23,10.89,0.0,32.36
297,2021-11-23 09:55:00+00:00,6.76,22.35,0.06,37.25,13.71,15.64,4.11,12.03,12.25,23.85,0.0,18.04


In [9]:
se = Sensor(31035)
sensor_data = se.parent.get_historical(weeks_to_get=1,
                              thingspeak_field='secondary')[['created_at', '2.5um/dl']]
sensor_data['created_at'] = sensor_data['created_at'].dt.floor('T')
sensor_data

,created_at,2.5um/dl
entry_id,,
362673,2021-11-24 22:41:00+00:00,5.27
362674,2021-11-24 22:43:00+00:00,2.31
362675,2021-11-24 22:45:00+00:00,6.05
362676,2021-11-24 22:47:00+00:00,3.78
362677,2021-11-24 22:49:00+00:00,3.69
...,...,...
366304,2021-11-29 23:50:00+00:00,12.09
366305,2021-11-29 23:52:00+00:00,12.80
366306,2021-11-29 23:54:00+00:00,6.03


In [10]:
se = Sensor(99769)
sensor_data = se.parent.get_historical(weeks_to_get=1,
                              thingspeak_field='secondary')[['created_at', '2.5um/dl']]
sensor_data['created_at'] = sensor_data['created_at'].dt.floor('T')
sensor_data

,created_at,2.5um/dl
entry_id,,
52511,2021-11-23 00:00:00+00:00,7.07
52512,2021-11-23 00:02:00+00:00,11.05
52513,2021-11-23 00:04:00+00:00,15.70
52514,2021-11-23 00:06:00+00:00,10.74
52515,2021-11-23 00:08:00+00:00,16.90
...,...,...
57530,2021-11-29 23:50:00+00:00,17.61
57531,2021-11-29 23:52:00+00:00,12.59
57532,2021-11-29 23:54:00+00:00,12.25


In [11]:
houston_data = df.loc[(df["lat"] >= 29) & (df["lat"] <= 30) & (df["lon"] >= -96) & (df["lon"] <= -94)]
outside = houston_data[houston_data['location_type'] == 'outside']
sensors = outside.index.to_list()
time_series_index(sensors)

,created_at,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,...,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y
0,2021-11-23 00:01:00+00:00,2.82,5.09,0.00,7.07,12.47,5.19,3.43,6.43,8.28,...,3.33,7.63,5.10,3.04,3.07,3.14,2.26,3.04,3.99,9.39
42,2021-11-23 01:25:00+00:00,4.79,15.45,0.22,7.52,22.07,4.21,4.76,4.61,7.45,...,5.15,7.09,7.75,7.41,3.73,13.12,0.00,0.73,4.35,11.39
44,2021-11-23 01:29:00+00:00,4.71,14.18,0.24,13.59,23.09,6.22,4.78,2.95,6.75,...,2.82,7.59,8.93,7.26,4.75,17.22,0.00,1.79,3.52,5.24
46,2021-11-23 01:33:00+00:00,3.11,16.35,0.00,13.82,26.11,5.67,2.98,5.02,6.61,...,4.98,10.79,8.93,11.36,4.81,21.15,1.98,3.91,4.14,7.25
47,2021-11-23 01:35:00+00:00,4.58,17.09,0.00,20.85,23.72,7.89,5.75,3.37,9.03,...,3.61,11.04,4.74,8.51,5.46,18.30,0.00,2.99,5.14,7.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,2021-11-23 06:23:00+00:00,8.21,37.75,0.00,61.10,25.73,13.78,5.22,9.54,10.00,...,11.18,17.78,7.04,16.95,3.38,18.98,0.62,3.83,4.38,18.23
192,2021-11-23 06:25:00+00:00,7.04,37.10,0.00,59.77,26.44,12.00,2.91,12.42,11.25,...,9.82,20.39,4.53,15.38,5.12,23.38,0.20,4.49,4.62,15.70
194,2021-11-23 06:29:00+00:00,5.05,31.67,0.58,58.82,28.70,13.82,6.20,6.78,8.24,...,10.14,21.24,10.86,19.57,5.36,21.79,0.00,4.56,4.71,20.50
195,2021-11-23 06:31:00+00:00,8.62,38.84,0.29,53.11,29.00,13.31,8.76,7.12,10.14,...,11.70,13.61,10.25,11.53,4.67,23.87,0.00,4.00,4.61,17.02
